In [1]:
from sklearn.naive_bayes import MultinomialNB # ideal für counting features wie bow oder tfidf https://towardsdatascience.com/why-how-to-use-the-naive-bayes-algorithms-in-a-regulated-industry-with-sklearn-python-code-dbd8304ab2cf
from sklearn.naive_bayes import GaussianNB # für Features in Decimal Form geeignet
from sklearn.naive_bayes import ComplementNB # ähnlich wie Multinomial, soll sich aber besser für imbalanced data eignen
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    recall_score,
    precision_score,
    classification_report,
    matthews_corrcoef, 
)
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
import joblib

In [2]:
def evaluate(y_test,y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    print("F1 Score:", f1)
    print("Recall:", recall)
    print("Precision:", precision)
    print("Accuracy:", accuracy)
    print("MCC:", mcc)
    print("\n")
    print("Confusion Matrix")
    print(pd.DataFrame(confusion_matrix(y_test, y_pred)))
    print("\n")
    print("Classification Report")
    print(classification_report(y_test, y_pred))

## Evaluation neue Vectorize-Funktionen (08.12.)

In [3]:
%run ../../functions/vectorize_functions.py

In [4]:
filepath_name = (('../../../data/mixed_dataset/train_cleaned.csv'))
df_cleaned = pd.read_csv(filepath_name, encoding='utf-8')

In [5]:
X_train, X_test, y_train, y_test, vectorizer_glv = vectorize_glove(df=df_cleaned, text_column='tweet_cleaned', 
                                                                                 label_column="label")

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(type(X_train))

(57332, 100)
(24572, 100)
(57332,)
(24572,)
<class 'numpy.ndarray'>


In [9]:
# save vectorizer

joblib.dump(vectorizer_glv, 'saved_models/vectorizer/vectorizer_glv.joblib')

['saved_models/vectorizer/vectorizer_glv.joblib']

## GaussianNB

In [7]:
clf = GaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [8]:
evaluate(y_test, y_pred)

F1 Score: 0.43345615327929254
Recall: 0.6658365406384423
Precision: 0.32131541571069594
Accuracy: 0.6871235552661566
MCC: 0.28403597731749025


Confusion Matrix
       0     1
0  13943  6212
1   1476  2941


Classification Report
              precision    recall  f1-score   support

           0       0.90      0.69      0.78     20155
           1       0.32      0.67      0.43      4417

    accuracy                           0.69     24572
   macro avg       0.61      0.68      0.61     24572
weighted avg       0.80      0.69      0.72     24572



In [11]:
results_list = []

param_grid = {
    'priors' : [None,[.1,.9], [.25,.75], [.5,.5], [.75,.25],[.9,.1]],
    'var_smoothing' : [1.0e-10,1e-9, 0.01, 0.05, 0.1, 0.2, 0.4, 0.5, 0.6, 0.8, 1]
}

nb = GaussianNB()

grid_search = GridSearchCV(estimator=nb, param_grid=param_grid, scoring='f1', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

for idx, params in enumerate(grid_search.cv_results_['params']):
    model = GaussianNB(**params)  
    model.fit(X_train, y_train)  

    y_train_pred = model.predict(X_train)

    y_test_pred = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)
    train_mcc = matthews_corrcoef(y_train, y_train_pred)

    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    test_mcc = matthews_corrcoef(y_test, y_test_pred)

    result_dict = {
        'model': 'Glove',
        'priors': params['priors'],
        'var_smoothing': params['var_smoothing'],
        'train_f1': train_f1,
        'train_recall': train_recall,
        'train_precision': train_precision,
        'train_accuracy': train_accuracy,
        'train_mcc': train_mcc,
        'test_f1': test_f1,
        'test_recall': test_recall,
        'test_precision': test_precision,
        'test_accuracy': test_accuracy,
        'test_mcc': test_mcc
    }

    results_list.append(result_dict)

    filename = f'saved_models/nb_glv/model_nb_glv_{idx}.joblib'
    joblib.dump(model, filename)


results_df = pd.DataFrame(results_list)

results_df.to_csv('eval_data/nb_glv_mixed_dataset.csv', index=False)

print(results_df)


Fitting 3 folds for each of 66 candidates, totalling 198 fits
    model      priors  var_smoothing  train_f1  train_recall  train_precision  \
0   Glove        None   1.000000e-10  0.435458      0.665182         0.323675   
1   Glove        None   1.000000e-09  0.435458      0.665182         0.323675   
2   Glove        None   1.000000e-02  0.436510      0.694810         0.318212   
3   Glove        None   5.000000e-02  0.431096      0.740414         0.304068   
4   Glove        None   1.000000e-01  0.429654      0.744094         0.302024   
..    ...         ...            ...       ...           ...              ...   
61  Glove  [0.9, 0.1]   4.000000e-01  0.329260      0.267428         0.428283   
62  Glove  [0.9, 0.1]   5.000000e-01  0.257854      0.179609         0.456897   
63  Glove  [0.9, 0.1]   6.000000e-01  0.188518      0.117157         0.482264   
64  Glove  [0.9, 0.1]   8.000000e-01  0.090829      0.049768         0.519192   
65  Glove  [0.9, 0.1]   1.000000e+00  0.038346 

In [3]:
results_df.sort_values(by=['test_f1'],ascending=False)

,model,priors,var_smoothing,train_f1,train_recall,train_precision,train_accuracy,train_mcc,test_f1,test_recall,test_precision,test_accuracy,test_mcc
59,Glove,"[0.9, 0.1]",1.000000e-01,0.437298,0.636619,0.333029,0.704859,0.288313,0.439375,0.643197,0.333647,0.704949,0.291846
58,Glove,"[0.9, 0.1]",5.000000e-02,0.441296,0.664892,0.330239,0.696714,0.294637,0.438973,0.664025,0.327856,0.694897,0.291828
57,Glove,"[0.9, 0.1]",1.000000e-02,0.441361,0.636038,0.337929,0.709953,0.294066,0.436084,0.629839,0.333493,0.707187,0.287036
55,Glove,"[0.9, 0.1]",1.000000e-10,0.439160,0.612413,0.342317,0.718220,0.291001,0.434853,0.609011,0.338152,0.715448,0.285410
56,Glove,"[0.9, 0.1]",1.000000e-09,0.439160,0.612413,0.342317,0.718220,0.291001,0.434853,0.609011,0.338152,0.715448,0.285410
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Glove,"[0.9, 0.1]",5.000000e-01,0.257854,0.179609,0.456897,0.813751,0.198805,0.250000,0.174100,0.443228,0.812225,0.189124
63,Glove,"[0.9, 0.1]",6.000000e-01,0.188518,0.117157,0.482264,0.818304,0.168178,0.180197,0.112067,0.459610,0.816702,0.156040
10,Glove,NaN,1.000000e+00,0.169567,0.102827,0.483167,0.818566,0.157435,0.163242,0.098936,0.466382,0.817679,0.148625
64,Glove,"[0.9, 0.1]",8.000000e-01,0.090829,0.049768,0.519192,0.820519,0.116946,0.093369,0.051166,0.533019,0.821382,0.121906
